In [1]:
import funzioni02 as f2
import numpy as np
import qiskit.ignis.mitigation.measurement as mc
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt
from sympy import *

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_sim_jakarta = sim_noisy_jakarta 
backend = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

In [2]:

def gates(steps, tempo, precision):
    U = f2.Trotter_N_approx(steps=steps, tempo=tempo, precision=precision)

    ### considero la colonna relativa allo stato iniziale desiderato
    ### in questo caso |110>  !!!!!DEVO IMPLEMENTARE QUESTA PARTE PER TUTTI GLI STATI!!!!!!!

    b0=im(U[3*8+6])
    b1=im(U[5*8+6])
    alp0=re(U[3*8+6])
    alp1=re(U[5*8+6])
    alp2=re(U[6*8+6])

    ### calcolo i coefficienti per i gate a 2 qubit

    r1=acos(sqrt(alp2**2+b1**2))
    a1=f2.angolo(alp2,b1)
    a2=0
    phi1=np.pi+a2-f2.angolo(alp1,b1)
    r2=acos(sqrt(alp1**2+b1**2)/sin(r1))
    phi2=-phi1-f2.angolo(alp0,b0)

    a_1_1 = -float(- a1 + phi1 - np.pi)/2
    a_2_1 = float(a1 + phi1 - np.pi)/2
    a_3_1 = float(r1)

    a_1_2 = -float(- a2 + phi2 - np.pi)/2
    a_2_2 = float(a2 + phi2 - np.pi)/2
    a_3_2 = float(r2)

    qr1=QuantumRegister(2)
    qc1=QuantumCircuit(qr1)

    qc1.rz(a_1_1*2,qr1[1])
    qc1.h(qr1[0])
    qc1.cx(qr1[0],qr1[1])
    qc1.ry(-a_3_1,qr1[0])
    qc1.ry(-a_3_1,qr1[1])
    qc1.cx(qr1[0],qr1[1])
    qc1.h(qr1[0])
    qc1.rz(a_2_1*2,qr1[1])

    qr=QuantumRegister(2)
    qc=QuantumCircuit(qr)

    qc.rz(a_1_2*2,qr[1])
    qc.h(qr[0])
    qc.cx(qr[0],qr[1])
    qc.ry(-a_3_2,qr[0])
    qc.ry(-a_3_2,qr[1])
    qc.cx(qr[0],qr[1])
    qc.h(qr[0])
    qc.rz(a_2_2*2,qr[1])

    return qc1, qc


In [3]:
steps = 48
tempo = np.pi/17
precision = 40

gate_1, gate_2 = gates(steps, tempo, precision)

qr = QuantumRegister(3)
qc = QuantumCircuit(qr)

qc.x(qr[1])
qc.append(gate_1, [qr[0],qr[1]])
qc.barrier()
qc.x(qr[2])
qc.append(gate_2, [qr[1],qr[2]])


In [4]:
transpile(gate_1, basis_gates=["cx", "x", "rz", "sx"]).draw()

global phase: 3π/2
        ┌─────────┐  ┌────┐┌─────────┐     ┌────┐┌────────────┐┌────┐┌────────┐»
q0_0: ──┤ Rz(π/2) ├──┤ √X ├┤ Rz(π/2) ├──■──┤ √X ├┤ Rz(2.7763) ├┤ √X ├┤ Rz(-π) ├»
      ┌─┴─────────┴─┐└────┘└─────────┘┌─┴─┐├────┤├────────────┤├────┤├────────┤»
q0_1: ┤ Rz(-2.1378) ├─────────────────┤ X ├┤ √X ├┤ Rz(2.7763) ├┤ √X ├┤ Rz(-π) ├»
      └─────────────┘                 └───┘└────┘└────────────┘└────┘└────────┘»
«            ┌─────────┐  ┌────┐┌─────────┐
«q0_0: ──■───┤ Rz(π/2) ├──┤ √X ├┤ Rz(π/2) ├
«      ┌─┴─┐┌┴─────────┴─┐└────┘└─────────┘
«q0_1: ┤ X ├┤ Rz(1.3812) ├─────────────────
«      └───┘└────────────┘

In [5]:
qc.draw()

┌───────────────┐ ░                       
q2_0: ─────┤0              ├─░───────────────────────
      ┌───┐│  circuit-1034 │ ░      ┌───────────────┐
q2_1: ┤ X ├┤1              ├─░──────┤0              ├
      └───┘└───────────────┘ ░ ┌───┐│  circuit-1035 │
q2_2: ───────────────────────░─┤ X ├┤1              ├
                             ░ └───┘└───────────────┘

In [6]:
cal = f2.calibration_cirquit(type="column_evolution")
cal.draw()

┌───┐┌───┐┌────┐ ░ ┌────┐┌───┐ ░                ┌───┐┌───┐ ░ ┌───┐┌───┐»
q18_0: ┤ X ├┤ Y ├┤ √X ├─░─┤ √X ├┤ Y ├─░─────────────■──┤ X ├┤ Y ├─░─┤ X ├┤ Y ├»
       └───┘└───┘└────┘ ░ ├───┬┘├───┤ ░ ┌───┐┌───┐┌─┴─┐└───┘└───┘ ░ └───┘└───┘»
q18_1: ─────────────────░─┤ X ├─┤ Y ├─░─┤ X ├┤ Y ├┤ X ├───────────░───────────»
                        ░ └───┘ └───┘ ░ └───┘└───┘└───┘           ░           »
q18_2: ─────────────────░─────────────░───────────────────────────░───────────»
                        ░             ░                           ░           »
«            ┌───┐┌───┐┌───┐ ░ ┌───┐      ░                 ░                »
«q18_0: ──■──┤ X ├┤ Y ├┤ X ├─░─┤ Y ├──────░─────────────────░────────────────»
«       ┌─┴─┐└───┘└───┘└───┘ ░ ├───┤┌───┐ ░ ┌───┐┌───┐┌───┐ ░ ┌───┐┌───┐┌───┐»
«q18_1: ┤ X ├────────────────░─┤ X ├┤ Y ├─░─┤ X ├┤ Y ├┤ X ├─░─┤ Y ├┤ X ├┤ Y ├»
«       └───┘                ░ └───┘└───┘ ░ └───┘└───┘└───┘ ░ ├───┤├───┤└───┘»
«q18_2: ─────────────────────░────────────░─────────────────░─┤ X ├┤ Y ├─────»
«                            ░            ░                 ░ └───┘└───┘     »
«        ░                           ░                           ░           »
«q18_0: ─░───────────────────────────░───────────────────────────░───────────»
«        ░                ┌───┐┌───┐ ░ ┌───┐┌───┐     ┌───┐┌───┐ ░ ┌───┐┌───┐»
«q18_1: ─░─────────────■──┤ X ├┤ Y ├─░─┤ X ├┤ Y ├──■──┤ X ├┤ Y ├─░─┤ X ├┤ Y ├»
«        ░ ┌───┐┌───┐┌─┴─┐└───┘└───┘ ░ └───┘└───┘┌─┴─┐└───┘└───┘ ░ ├───┤├───┤»
«q18_2: ─░─┤ X ├┤ Y ├┤ X ├───────────░───────────┤ X ├───────────░─┤ X ├┤ Y ├»
«        ░ └───┘└───┘└───┘           ░           └───┘           ░ └───┘└───┘»
«        ░           
«q18_0: ─░───────────
«        ░           
«q18_1: ─░───────────
«        ░ ┌───┐┌───┐
«q18_2: ─░─┤ X ├┤ Y ├
«        ░ └───┘└───┘

In [7]:
cal = f2.calibration_cirquit(type="column_evolution_remake")
cal.draw()



┌───┐┌────┐ ░ ┌────┐     ┌───┐┌────┐ ░ ┌────┐     ┌───┐┌────┐ ░ ┌────┐»
q19_0: ┤ X ├┤ √X ├─░─┤ √X ├──■──┤ X ├┤ √X ├─░─┤ √X ├──■──┤ X ├┤ √X ├─░─┤ √X ├»
       ├───┤└────┘ ░ ├───┬┘┌─┴─┐├───┤├────┤ ░ ├────┤┌─┴─┐├───┤└────┘ ░ ├───┬┘»
q19_1: ┤ X ├───────░─┤ X ├─┤ X ├┤ X ├┤ √X ├─░─┤ √X ├┤ X ├┤ X ├───────░─┤ X ├─»
       └───┘       ░ └───┘ └───┘└───┘└────┘ ░ └────┘└───┘└───┘       ░ └───┘ »
q19_2: ────────────░────────────────────────░────────────────────────░───────»
                   ░                        ░                        ░       »
«        ░             ░                        ░                        ░ »
«q19_0: ─░─────────────░────────────────────────░────────────────────────░─»
«        ░ ┌───┐┌────┐ ░ ┌────┐     ┌───┐┌────┐ ░ ┌────┐     ┌───┐┌────┐ ░ »
«q19_1: ─░─┤ X ├┤ √X ├─░─┤ √X ├──■──┤ X ├┤ √X ├─░─┤ √X ├──■──┤ X ├┤ √X ├─░─»
«        ░ ├───┤└────┘ ░ ├───┬┘┌─┴─┐├───┤├────┤ ░ ├────┤┌─┴─┐├───┤└────┘ ░ »
«q19_2: ─░─┤ X ├───────░─┤ X ├─┤ X ├┤ X ├┤ √X ├─░─┤ √X ├┤ X ├┤ X ├───────░─»
«        ░ └───┘       ░ └───┘ └───┘└───┘└────┘ ░ └────┘└───┘└───┘       ░ »
«             
«q19_0: ──────
«       ┌────┐
«q19_1: ┤ √X ├
«       ├───┬┘
«q19_2: ┤ X ├─
«       └───┘

In [8]:
f2.matrix_from_cirquit(cal, phase=0)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


Matrix([
[1.0,   0,   0,   0,   0,   0,   0,   0],
[  0, 1.0,   0,   0,   0,   0,   0,   0],
[  0,   0, 1.0,   0,   0,   0,   0,   0],
[  0,   0,   0, 1.0,   0,   0,   0,   0],
[  0,   0,   0,   0, 1.0,   0,   0,   0],
[  0,   0,   0,   0,   0, 1.0,   0,   0],
[  0,   0,   0,   0,   0,   0, 1.0,   0],
[  0,   0,   0,   0,   0,   0,   0, 1.0]])